<img src="squid2.jpg" width="200">
<h1><span style="Color: red">S</span>age <span style="Color: red">Qui</span>z <span style="Color: red">D</span>eveloper
</h1>

Welcome to Squid! An object-oriented approach to creating variants of quiz questions.

Squid was created by Florian Breuer (florian.breuer@newcastle.edu.au).

This code can be found on Github: https://github.com/florianbreuer/Squid

<hr>

You are looking at a quick and dirty introduction to the main Squidd features. A more comprehensive tutorial will come later.
Can't see the whole page? Reduce the magnification in your browser with "<tt>ctrl -</tt>".

Squid runs in the Sagemath Jupyter notebook. To use it, install Sage (for free!) from https://www.sagemath.org/

Make sure you have the following files in one directory:
<ul>
    <li><a href="https://florianbreuer.github.io/florianbreuer/Squid/Squid.ipynb" download="Squid.ipynb">Squid.ipynb</a>        (this notebook) </li>
    <li><a href="https://florianbreuer.github.io/florianbreuer/Squid/QTI_template.xml" download="QTI_template.xml"> QTI_template.xml</a>   (a template file needed for creating QTI output) </li>
    <li><a href="https://florianbreuer.github.io/florianbreuer/Squid/Squid-tools2.sage" download="Squid-tools2.sage">Squid-tools2.sage</a>  (the main Sage code for Squid).</li>
</ul>
    
Then run the notebook server and open Squid.ipynb

First, we load the basic classes and tools:

In [5]:
load("Squid-tools2.sage")

Squid Tools v2, 11 October 2021
By Florian Breuer, florian.breuer@newcastle.edu.au

Imported the following functions:
  For handling QTI files:
   id_generator, initialise_qti, qti_set_question_text, qti_MCQ_new, qti_set_points, qti_set_identifier,
    qti_insert_question, qti_set_question_text, qti_file_upload_question_new, ET_MCQ, ET_file_upload_question, save_qti

  For nicer typesetting:
   nicify, nicify0, plus, pplus, suppress1, Taylor

  For manipulating and typesetting matrices:
   scramble, scramble_full, tootrivial, latexdet

  For handling LMS files and marking schemes:
   SaveToBBfile, SaveToQtiFile, PrintMarkingScheme, TypesetMarkingScheme, SaveMarkingScheme

Imported the following classes:
  MATHJAX
  linear_system
  Question_written
  Question_MCQ
  QuestionPool


Squid currently only supports two question types: Multiple-choice and Written-answer. 
Squid defines two classes, <tt>Question_MCQ</tt> and <tt>Question_written</tt>. 

to create a new multiple-choice question, we create a subclass of <tt>Question_MCQ</tt> which is initialised by various parameters that can vary from one question variant to another:

In [15]:
class MyMCQ(Question_MCQ):
    def __init__(self,a,b):
        Question_MCQ.__init__(self)  # this loads some useful methods from Question_MCQ
        
        self.question_text = f'Compute ${a} + {b}$:'
        
        self.answer = str(a+b)
        
        self.wrong_answers = [str(a+b+1), str(a+b+2), str(a+b-1)]
        
Q = MyMCQ(3,5)
display(Q)

Compute $3 + 5$:

Now we have a working question prototype, let's make a list of variants:

In [17]:
ablist = [3,4,5,6,7,8]  # parameter values

L = [MyMCQ(a,b) for a in ablist for b in ablist if a<b]

for Q in L:
    display(Q)

Compute $3 + 4$:

Compute $3 + 5$:

Compute $3 + 6$:

Compute $3 + 7$:

Compute $3 + 8$:

...

Compute $5 + 7$:

Compute $5 + 8$:

Compute $6 + 7$:

Compute $6 + 8$:

Compute $7 + 8$:

If we're happy with this, we can export this list of variants into QTI format for Canvas. The result will be a .zip file which can be imported to Canvas (under "settings"). Canvas will turn it into an assessment (which you can delete) and a question pool, which is what we wanted.

In [18]:
SaveToQtiFile(L, 'UploadMeToCanvas')

Got the multiple-choice question: Question 42
Got the file upload question: Question 69
Created UploadMeToCanvas.xml
Created UploadMeToCanvas.zip - You can upload this file to Canvas.


Of course, our simple addition question is far too trivial. Here's an example of a more complicated question, from MATH1120.

In [21]:
x,y = var('x y')

class DirectionalDerivative(Question_MCQ):
    def __init__(self,f,V,P,variant_number=0):
        '''Compute the directional derivative of f(x,y) at the point P in the direction v.'''
        Question_MCQ.__init__(self)
        self.variant_number = variant_number
        
        fx = diff(f,x)
        fy = diff(f,y)
        
        u = (V[0]/sqrt(V[0]^2+V[1]^2),V[1]/sqrt(V[0]^2+V[1]^2))
        
        self.question_text = r"""Compute the directional derivative of \(f(x) = """\
        +latex(f)+\
        r"""\) at the point \(P = """\
        +latex(P)+\
        r"""\) in the direction \(\langle"""\
        +latex(V[0])+','+latex(V[1])+\
        r"""\rangle\)."""
        
        self.answer = r'\('+latex(u[0]*fx(x=P[0],y=P[1]) + u[1]*fy(x=P[0],y=P[1]))+r'\)'
        
        self.wrong_answers = [r'\('+latex(V[0]*fx(x=P[0],y=P[1]) + V[1]*fy(x=P[0],y=P[1]))+r'\)',
                              r'\('+latex(V[0]*fx(x=P[0],y=P[1]) - V[1]*fy(x=P[0],y=P[1]))+r'\)',
                              r'\('+latex(u[0]*fy(x=P[0],y=P[1]) + u[1]*fx(x=P[0],y=P[1]))+r'\)',
                             ]

# define some parameters to use
flist = [x^2+y, x+y^2, x^3-y, y^3-x]
Plist = [(1,2),(1,-2),(-1,2),(2,1),(2,-1)]

# Now we create a list of question variants with combinations of the above parameters:
L = [DirectionalDerivative(f,V,P) for f in flist for V in Plist for P in Plist if (V!=P)]

# we remove any variants for which the various answers aren't distinct:
L = [Q for Q in L if Q.has_distinct_answers()]

display(L[0])

Compute the directional derivative of \(f(x) = x^{2} + y \) at the point \(P = \left(1, -2\right) \) in the direction \(\langle 1 , 2 \rangle\).

This creates 74 variants. Some may me more challenging than others, so we really want to look at the variants, select a subset of them to use and upload only those. 

For this we create a question pool as an instance of the <tt>QuestionPool</tt> class and use its <tt>selection_wizard</tt> to
run an interactive tool that lets us select the variants we want and export them.

In [20]:
MCQpool = QuestionPool(L, quiz_name = 'Week6', question_name='CA6_DirectionalDerivative')
MCQpool.selection_wizard()

Output(layout=Layout(border='1px solid black'))

Button(description='Clear Output', style=ButtonStyle())

We can also make written-answer questions. Now there's more work to do, since the model solution must also be coded. The final output will be a LaTeX file containing the model solutions to each variant. This can then be compiled to PDF. 

The marking scheme is created with 

<tt>SaveMarkingScheme(L, filename)</tt>, 

or you just click the button in the wizard.

For example, the marking scheme for the first 10 variants of the following question (from MATH1120) is: 
<a href="https://florianbreuer.github.io/florianbreuer/MATH1120-S2-2021-Week-8-MarkingScheme.pdf" target="_blank">MATH1120-S2-2021-Week-8-MarkingScheme.pdf</a>.

In [24]:
class LA6_DEsystem_written(Question_Written):
    
    def __init__(self, v1, v2, l1, l2, variant_number=0):
        '''2x2 system of coupled DEs, with eigenvectors v1, v2 and eigenvalues l1, l2.'''
        Question_Written.__init__(self)
        if l1==l2:
            raise ValueError('Eigenvalues must be distinct')
        if v1[0]*v2[1]==v1[1]*v2[0]:
            raise ValueError('Eigenvectors must be linear independent')
        P = matrix(2,2,[v1[0],v2[0],v1[1],v2[1]])
        D = matrix(2,2,[l1,0,0,l2])
        A = P*D*P.inverse()
        self.A = A
        sys = r'\begin{align*}'+'\n'\
            +r'\frac{dx_1}{dt} & = '+suppress1(A[0][0])+'x_1'+plus(A[0][1])+r'x_2\\'+'\n'\
            +r'\frac{dx_2}{dt} & = '+suppress1(A[1][0])+'x_1'+plus(A[1][1])+r'x_2'+'\n'\
            +r'\end{align*}'
        
        la = var('lambda_')
        M1 = A - l1*identity_matrix(2)
        M2 = A - l2*identity_matrix(2)
        I1 = matrix(2,1,[0,0])
        v1 = matrix(2,1,v1)
        v2 = matrix(2,1,v2)
        
        sol = r'C_1e^{'+suppress1(l1)+r't}'+latex(v1)+r' + C_2e^{'+suppress1(l2)+r't}'+latex(v2)
        
        self.question_text_basic = 'Solve the following system of coupled linear '+\
        'differential equations, showing all your work:\n'+sys

        self.solution_text = 'The coefficient matrix is\n'+\
        r'\[ A = '+'\n'+latex(A)+'.\n'+r'\]'+'\n'+\
        'We determine its eigenvalues and eigenvectors:\n'+\
        r'\['+'\n'+\
        r'0 = \det(A-\lambda I) = '+latexdet(A-la*identity_matrix(2))+' = '+\
        latex(det(A-la*identity_matrix(2)))+\
        r' = (\lambda '+pplus(-l1)+r')(\lambda '+pplus(-l2)+').\n'+\
        r'\]'+'\n'+\
        r'Thus \(\lambda_1 = '+latex(l1)+r'\), and \(\lambda_2 = '+latex(l2)+r'\).'+\
        'Next, we determine the corresponding eigenvectors.\n\n'+\
        r'For \(\lambda_1 = '+latex(l1)+r'\), we solve the system'+'\n'+\
        r'\['+'\n'+\
        latex(M1.augment(I1, subdivide=true))+r'\sim'+\
        latex(M1.echelon_form().augment(I1, subdivide=true))+'\n'+\
        r'\]'+'\n'+\
        r'\['+'\n'+\
        r' \Longrightarrow \; {\bf v}_1 = '+latex(v1)+'.\n'+\
        r'\]'+'\n'+\
        r'For \(\lambda_2 = '+latex(l2)+r'\), we solve the system'+'\n'+\
        r'\['+'\n'+\
        latex(M2.augment(I1, subdivide=true))+r'\sim'+\
        latex(M2.echelon_form().augment(I1, subdivide=true))+'\n'+\
        r'\]'+'\n'+\
        r'\['+'\n'+\
        r' \Longrightarrow \; {\bf v}_2 = '+latex(v2)+'.\n'+\
        r'\]'+'\n\n'+\
        'The final solution is thus\n'+\
        r'\['+'\n'+\
        r'\left(\begin{array}{l} x_1(t) \\ x_2(t)\end{array}\right) = '+'\n'+sol+'.\n'+\
        r'\]'
        
        self.table_header = ['A', 'Solution']
        
        self.table_row = ['$'+latex(A)+'$', '$'+sol+'$']
        
def is_int_matrix(A):
    for x in A.list():
        if frac(x)!=0:
            return(false)
    return(true)
        
Q = LA6_DEsystem_written((1,1),(1,2),1,-3)
Q.update_variant_number(23)
display(Q)
Q.test_solution_page('test.tex')

llist = [-3,-2,2,3]
vlist = [(1,1), (-1,1), (2,1), (1,2), (-2,1), (1,-2)]

L = []
for l1 in llist:
    for l2 in llist:
        if l1<l2:
            for v1 in vlist:
                for v2 in vlist:
                    if v1[0]*v2[1]!=v1[1]*v2[0]:
                        Q = LA6_DEsystem_written(v1, v2, l1, l2)
                        if is_int_matrix(Q.A) and max(abs(x) for x in Q.A.list()) < 6\
                        and min(abs(x) for x in Q.A.list()) > 0:
                            L.append(Q)
                            

pool = QuestionPool(L, quiz_name = 'Week-8', question_name='DEsystem_written18-test')
pool.selection_wizard()   

Solve the following system of coupled linear differential equations, showing all your work:
 \begin{align*}
\frac{dx_1}{dt} & = 5 x_1 - 4 x_2\\ 
 \frac{dx_2}{dt} & = 8 x_1 - 7 x_2 
 \end{align*} <br>[For office use only: V 23 ]

Output(layout=Layout(border='1px solid black'))

Button(description='Clear Output', style=ButtonStyle())

That's the end of our quick tour. If you want to know more, contact me at florian.breuer@newcastle.edu.au.

In the future, I hope to add more question types as well as functionality to create random variants of paper quizzes for when we go back to invigilated in-person tests. Watch this space!